### Notebook for experimental testing of the extraction steps 

The workflow manager provides all methods to test each extraction step, the PromptsReader can be used for read in process or prompts texts. 

In [1]:
from WorkflowManager import WorkflowManager, ModelChoice
import PromptsReader

workflow_manager = WorkflowManager()
process = PromptsReader.read_text_file(PromptsReader.TextFiles.PROCESS_1.value)
## extract activity-labels from the input process
activities_labeled, is_valid = workflow_manager.extract_activities(process, model_choice=ModelChoice.GPT_3_5_TURBO)

# Check the results
if is_valid:
    print("Extracted activities:", activities_labeled)
else:
    print("JsonError - please be more specific in your process description and follow the recommendations.")

In the context of business process management, you are tasked with extracting process activities from a given text. A process activity is a unit of work or a specific step in a process, usually performed by an individual or a group. In the following task please list the extracted activities as response in json format: {"activity-labels": ["activity1", "activity2", ... ]}. Please do not add any other information. ### instruction: Person A sends an email. Person B reads the email. If relevant, Person B forwards the mail.  ### response: {"activity-labels": ["sends an email", "reads the email", "forwards the mail"]} ### instruction: A Customer places an order. Then the Warehouse checks the payment of the order. If the payment is successful, the warehouse ships the order. Otherwise the warehouse rejects the order. Please provide the plain JSON output following the template and guidelines. and do not add any
information that is not described here and do not add any explanations, only show th

In [2]:
## extract actors from the input process and assign them to the activities
## input variables: process, activities_labeled
act_actors_labeled = workflow_manager.assign_actors(process, activities_labeled, model_choice=ModelChoice.GPT_3_5_TURBO)
print("Activities with actors:", act_actors_labeled)

In the context of business process management, you are tasked to assign activities to its performers. Process Participant:
A process participant is any individual or entity that participates in a business process. This could include individuals who initiate the process, those who respond to it, or those who are affected by it.
Activity:
An activity is a unit of work that can be performed by an individual or a group. It is a specific step in the process. One actor can perform one or more activties. If the actor was already assigned, please use the same id for assigning. 
Please just answer in the JSON format {"Activities": [{"activity-label": "", "actor-label": "", "actor-id": ""}], ..}. Please do not add any other explainations or text, only provide the plain JSON. 

Please note that named actors can be departments as well as employees of a department. The unique actor is still the department, no distinction is necessary. 

### instruction:
Process Model: 
The Sales Department requests

In [3]:
## Maps the activities to the Activity class for further processing
import Activity
activities_obj = Activity.Activities()
activities_mapped = activities_obj.from_label_json(act_actors_labeled)
print (activities_mapped.to_json())

[{"id": "T1", "label": "Customer places an order", "actor_label": "The Customer", "actor_id": "A1"}, {"id": "T2", "label": "Warehouse checks the payment of the order", "actor_label": "The Warehouse", "actor_id": "A2"}, {"id": "T3", "label": "Warehouse ships the order", "actor_label": "The Warehouse", "actor_id": "A2"}, {"id": "T4", "label": "Warehouse rejects the order", "actor_label": "The Warehouse", "actor_id": "A2"}]


In [4]:
## Create workflow with extracted activities as input (can last up to 1-2 minutes)
## Activities have to be mapped to json first
## If possible, also a diagram is created on the default path (see Process_Map_Creator.py)) 
workflow = workflow_manager.create_workflow_with_input(process, activities_mapped.to_json(), model_choice="gpt-3.5-turbo")

print (workflow) 

Assume the following for the definition of Workflow Models: A Workflow Model consists of the following node types: Task, XOR-Split, XOR-Join, Start-Node, End-Node. Nodes of type Start-Node have exactly one successor node and no predecessor node. Nodes of type End-Node have exactly one predecessor node and no successor node. Nodes of type Task (=Activities) have exactly one successor node and exactly one predecessor node. Tasks can be followed by gateways (XOR-Split or XOR-Join) or by other tasks. Nodes of type XOR-Split have one predecessor node and two successor nodes (successor_true, successor_false) and a boolean condition. Nodes of type XOR-Join have two predecessor nodes (predecessor_true, predecessor_false) and one successor node. I.e., the predecessor nodes of an XOR-Join, having this XOR-Join node as a successor node. If a process contains a XOR-Split gateway, it must also contain a XOR-Join gateway. 
### instruction: 
The MPON sends the dismissal to the MPOO. The MPOO reviews 

2023-10-15 21:40:22 [INFO] : Took [196.69ms] to generate 'generated_processes/process-XXX.png' diagram


{
  "WorkflowModel": {
    "start_node": {
      "id": "S1",
      "actor_label": "Customer",
      "actor_id": "A1"
    },
    "end_node": {
      "id": "E1",
      "actor_label": "Warehouse",
      "actor_id": "A2"
    },
    "tasks": [
      {
        "id": "T1",
        "label": "Customer places an order",
        "successor": "T2",
        "predecessor": "S1",
        "actor_label": "Customer",
        "actor_id": "A1"
      },
      {
        "id": "T2",
        "label": "Warehouse checks the payment of the order",
        "successor": "XS1",
        "predecessor": "T1",
        "actor_label": "Warehouse",
        "actor_id": "A2"
      },
      {
        "id": "T3",
        "label": "Warehouse ships the order",
        "successor": "XJ1",
        "predecessor": "XS1",
        "actor_label": "Warehouse",
        "actor_id": "A2"
      },
      {
        "id": "T4",
        "label": "Warehouse rejects the order",
        "successor": "XJ1",
        "predecessor": "XS1",
        "a